## Fine Tuning

#### Introduction

Fine-tuning lets you get more out of the models available through the API by providing:

Higher quality results than prompting Ability to train on more examples than can fit in a prompt Token savings due to shorter prompts Lower latency requests

OpenAI's text generation models have been pre-trained on a vast amount of text. To use the models effectively, we include instructions and sometimes several examples in a prompt. Using demonstrations to show how to perform a task is often called "few-shot learning."

Fine-tuning improves on few-shot learning by training on many more examples than can fit in the prompt, letting you achieve better results on a wide number of tasks. Once a model has been fine-tuned, you won't need to provide as many examples in the prompt. This saves costs and enables lower-latency requests.

At a high level, fine-tuning involves the following steps:

Prepare and upload training data Train a new fine-tuned model Evaluate results and go back to step 1 if needed Use your fine-tuned model


#### When to use fine-tuning

Fine-tuning OpenAI text generation models can make them better for specific applications, but it requires a careful investment of time and effort. We recommend first attempting to get good results with prompt engineering, prompt chaining (breaking complex tasks into multiple prompts), and function calling, with the key reasons being:

There are many tasks at which our models may not initially appear to perform well, but results can be improved with the right prompts - thus fine-tuning may not be necessary
Iterating over prompts and other tactics has a much faster feedback loop than iterating with fine-tuning, which requires creating datasets and running training jobs
In cases where fine-tuning is still necessary, initial prompt engineering work is not wasted - we typically see best results when using a good prompt in the fine-tuning data (or combining prompt chaining / tool use with fine-tuning)

In [1]:
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv('C:/Code_Apps/Learn-Generative-AI/03_chatgpt/.env'))

api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI
client = OpenAI()

### Upload file for fine-tuning

In [3]:
file_name = "data.jsonl"

file_data = open(file_name,'rb')

file_to_upload = client.files.create(
    file=file_data,
    purpose="fine-tune"
)
file_to_upload

FileObject(id='file-lmwkiXgcjeSxRL1bDhsy7OGI', bytes=3609, created_at=1708240890, filename='data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

### Create a fine-tune model

In [4]:
fine_tune_model = client.fine_tuning.jobs.create(
  training_file= file_to_upload.id,
  model="gpt-3.5-turbo"
)
fine_tune_model

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

In [ ]:
import time

while True:
    data = data = client.fine_tuning.jobs.retrieve(fine_tune_model.id)

    if data.status == "succeeded":
        break

    elif data.status == "failed":
        print("fine-tuning has been failed")

    else:
        print("fine-tune is still running")
        time.sleep(5)

        

In [ ]:
from openai.types.chat.chat_completion import ChatCompletion

def chat_completion(prompt:str)->str{
    response: ChatCompletion = client.chat.completions.create(
        model=data.fine_tuned_model,
        messages=[
            {"role": "system", "content": "You are a sarcastic assistant."},
            {
             "role": "user", "content": prompt
            }
        ]
    )

    return response.choices[0].messages.content
}

In [ ]:
chat_completion("Who is the child landed on space?")